In [1]:
cd ..

C:\Users\johan\github\designresolver


In [2]:
from autograd import jacobian, grad
import autograd.numpy as anp
import numpy as np
from itertools import chain

In [3]:
from execution import Component

In [11]:
A = np.random.rand(2,2)
b = np.random.rand(2)
def fx(x,y):
    return [y**2, b*x]
inputs = (1,2)
indims = (1,1)
outputs = (3,4)
outdims = (1,2)

In [12]:
c = Component(fx, inputs, outputs, 1, indims, outdims)

In [13]:
c2 = Component(lambda x,y: x**2+y**2, (1,2), component=1)

In [14]:
c2.indims

(1, 1)

In [15]:
c2.gradient(1,2)

{1: array(2.), 2: array(4.)}

In [16]:
c.graddict({1:3, 2:4})

{(3, 1): array(0.),
 (3, 2): array(8.),
 (4, 1): array([0.60726252, 0.13402966]),
 (4, 2): array([0., 0.])}

# Format of functions


Compliant

Options for inputs and outputs:
* List of arguments, where each argument has a size
* A numpy array that flattens all arguments 

Based on option 1) 

In [4]:
def fx(*x: list) -> list:
    return [sum(x)]

In [5]:
inputs = (1,2,3)
indims = (1,1,4)
outputs = (4,5)
outdims = (4,2)

In [6]:
def fx(a,b,c):
    return [a+2*b+3*c] #np.array((1,2))

In [265]:
A = np.random.rand(2,2)
b = np.random.rand(2)
def fx(x):
    return [x**2]
inputs = (1,)
indims = (1,)
outputs = (2,)
outdims = (1,)

In [273]:
xin = np.array((1.,2.))
xin = 1.

In [274]:
inargs = [xin]

In [275]:
f = lambda x: anp.hstack(fx(*anp.split(x, anp.cumsum(indims[:-1]))))
g = jacobian(f)

In [276]:
ins

array([1., 2.])

In [277]:
ins = np.hstack(inargs).astype(float)
jout = g(ins)

In [278]:
jout

array([[2.]])

In [279]:
{(outvr, invr):grad for outvr,grads in zip(outputs, np.split(jout, outdims[:-1], axis=0)) for invr,grad in zip(inputs, np.split(grads, np.cumsum(indims[:-1]), axis=1))}

{(2, 1): array([[2.]])}

In [310]:
tuple(range(3))

(0, 1, 2)

In [6]:
def generate_grad(fx, inputs, outputs, indims, outdims):
    f = lambda x: anp.hstack(fx(*anp.split(x, anp.cumsum(indims[:-1]))))
    g = jacobian(f)
    def getgrad(*inargs):
        ins = np.hstack(inargs).astype(float)
        jout = g(ins)
        outsplit = zip(outputs, np.split(jout, outdims[:-1], axis=0))
        insplit = lambda grads: zip(inputs, np.split(grads, np.cumsum(indims[:-1]), axis=1))
        return {(outvr, invr):np.squeeze(grad) for outvr,grads in outsplit for invr,grad in insplit(grads)}
    return getgrad

In [5]:
A = np.random.rand(2,2)
b = np.random.rand(2)
def fx(x):
    return [x**2, b*x]
inputs = (1,)
indims = (1,)
outputs = (2,3)
outdims = (1,2)

In [306]:
gfx = generate_grad(fx, inputs, outputs, indims, outdims)

In [309]:
gfx(1.0)

{(2, 1): array(2.), (3, 1): array([0.83731709, 0.85514987])}

In [9]:
c.function(1.0)

[1.0, array([0.76388878, 0.09116283])]

In [238]:
inargs = (1,1,np.array((1,2,3,4)))

In [236]:
fx(1,1,np.array((1,2,3,4)))

[array([3, 4, 5, 6]), array([1, 2])]

In [212]:
fx1 = lambda a,b,c: [a+b+c]
fx2 = lambda *x: [x[0]*x[1]]
fx3 = lambda x: [x+1]

In [225]:
f = lambda x: x[0]+x[1]
g = grad(f)
g(anp.array([1.,2.]), 1.0)

array([1., 1.])

Non compliant

In [213]:
fx4 = lambda a,b,c: a+b+c # output is not list
fx5 = lambda x: [x[0]*x[1]] # input would be a list

In [194]:
fx(np.array([1]))

array([2])

In [14]:
from compute import ureg
from unitutils import get_unit_multiplier

In [4]:
fx = lambda x: x[0]+x[1]+x[2]
g = grad(fx)

In [5]:
vector_fun = lambda x: anp.array([x[0]+x[1]+x[2], x[0]+x[2]])

x = np.random.randn(3)
jac = jacobian(vector_fun)(x)

In [6]:
jac

array([[1., 1., 1.],
       [1., 0., 1.]])

# Components
Different examples of using components

## Plain

In [7]:
vector_fun = lambda *x: anp.array([(x[0]+x[1])*x[2], x[0]+x[1]])

In [8]:
# Inputs should not be same as outputs!
inputs = (1,2,3)
outputs = (4,5)
c1 = Component(vector_fun, inputs, outputs, 1)

In [9]:
c1.jacobian(np.array([2.,4.,3.]))

array([[3., 3., 6.],
       [1., 1., 0.]])

In [10]:
c1.evaldict({1: 2, 2:4, 3:3})

array([18,  6])

## Inputs and outputs with units

In [143]:
inputs = (1,2,3) #The order is critical
# we describe inunits as a dictionary to decouple from inputs
inunits = {1:'kg', 2: 'g', 3:'m/s^2'}
outputs = (4,5)
outunits = {4:'N', 5:'g'}
outputs = (4,)
outunits = {4:'N'}

In [153]:
vector_fun = lambda *x: [(x[0]+x[1])*x[2], x[0]+x[1]]
vector_fun = lambda a,b,c: (a+b)*c

In [154]:
inpunitsflat = tuple(ureg(inunits[inpvar]) for inpvar in inputs)
outunitsflat = tuple(ureg(outunits[outvar]) for outvar in outputs)

In [155]:
expr_units = get_unit(vector_fun, inpunitsflat)
outunitpairs = tuple((outunit, outunitsflat[idx]) for idx, outunit in enumerate(expr_units))

In [156]:
expr_units

[1.0 <Unit('kilogram * meter / second ** 2')>]

In [157]:
convert, factors = unit_conversion_factors(outunitpairs, inpunitsflat)
convert, factors

(array([1.   , 0.001, 1.   ]), [1.0])

In [115]:
def flatten_list(ls):
    return ls if np.isscalar(ls)==1 else list(ls)

In [121]:
out = vector_fun(*(convert*np.array((1.,1.,1.)).flatten()))

In [127]:
1.001/1e-3

1000.9999999999999

In [125]:
out[1]/0.001

1000.9999999999999

In [136]:
vector_fun = lambda *x: [(x[0]+x[1])*x[2], x[0]+x[1]]

In [158]:
# What we want to make work
c1 = Component.withunits(lambda a,b,c: (a+b)*c, inputs, (4,), inunits, {4:'N'}, 1)

In [167]:
c1.graddict({1: 2., 2:1., 3:2.5})

C:\Users\johan\miniconda3\envs\mdo\lib\site-packages\autograd\tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


array([0., 0., 0.])

In [20]:
def expression_conversion_unit(expr_unit, tounit=None):
    unit = tounit if tounit else ureg('')
    if tounit:
        assert(unit.dimensionality == expr_unit.dimensionality)
        conversion_unit = unit
    else: # tounit was not given
        if not hasattr(expr_unit, 'units'):
            conversion_unit = ureg('')
        else:
            # if we evaluate an expression and get some crazy unit 
            # we bring it back to it's base dimensionality
            conversion_unit = ureg.Quantity(1, 
                expr_unit.to_base_units().units)
    return conversion_unit

In [21]:
def unit_conversion_factors(outunitpairs, inunits):
    convert = np.array([get_unit_multiplier(inp) for 
            inp in inunits])
    factors = []
    for outunit, tounit in outunitpairs:
        conversion_unit = expression_conversion_unit(outunit, tounit)
        factor = get_unit_multiplier(conversion_unit)
        factors.append(factor)
    return convert, factors

In [22]:
expression_conversion_unit(ureg.Quantity(1, 'km'))

1 <Unit('meter')>

In [23]:
unit_conversion_factors([(ureg.Quantity(1, 'g*m/s^2'), ureg.Quantity(1, 'mN'))], [ureg.Quantity(1, 'km'), ureg.Quantity(1, 'kg')])

(array([1000.,    1.]), [0.001])

In [24]:
from unitutils import MockFloat
import sympy as sp

In [25]:
def listify(out):
    return out if isinstance(out, list) else [out]

In [88]:
def get_unit(fx, inputunits):
    args = tuple(ureg.Quantity(MockFloat(1), inputunit) for inputunit in inputunits)
    dim = fx(*args)
    dims = listify(dim)
    # need this case if output is a float, which can happen when we have powers, e.g. 10^x:
    dims = [dim if isinstance(dim, ureg.Quantity) else ureg('') for dim in dims]
    return dims

In [96]:
get_unit(vector_fun, ('g', 'kg', 'm/s^2'))

[1.0 <Unit('gram * meter / second ** 2')>, 1.0 <Unit('gram')>]

In [27]:
get_unit(lambda *x: (x[0]+x[1])*x[2], ('g', 'kg', 'm/s^2'))

[1.0 <Unit('gram * meter / second ** 2')>]

In [28]:
def flatten_list(ls):
    return ls if np.isscalar(ls)==1 else list(ls)

In [29]:
def executable_with_conversion(convert, factors, fx):
    def scaled_fx(*args):
        return flatten_list(np.array(fx(*(convert*np.array(args).flatten())))/factors)
    return scaled_fx

In [30]:
from compute import Var

In [31]:
a = Var('a', 1, 'g')
b = Var('b', 2, 'kg')
c = Var('c', 3, 'm/s^2')

In [32]:
expr1 = (a+b)*c
expr2 = a-b

In [33]:
inputs = (a,b,c)
inpunitsflat = tuple(inpvar.varunit for inpvar in inputs)
fx = sp.lambdify(inputs, [expr1, expr2], anp) #anp_math
#outputs = (None,)
outunitsflat = (ureg.Quantity('N'),None)

In [34]:
inpunitsflat

(1 <Unit('gram')>, 1 <Unit('kilogram')>, 1.0 <Unit('meter / second ** 2')>)

In [35]:
def fx_with_units(fx, inunitsflat, outunitsflat):
    expr_units = get_unit(fx, inunitsflat)
    outunitpairs = tuple((outunit, outunitsflat[idx]) for idx, outunit in enumerate(expr_units))
    convert, factors = unit_conversion_factors(outunitpairs, inunitsflat)
    fx_scaled = executable_with_conversion(convert, factors, fx)
    return fx_scaled

In [36]:
fx_scaled = fx_with_units(fx, inpunitsflat, outunitsflat)

In [37]:
fx_scaled(1,2,3)

[6.003, -1.999]

In [ ]:
def fromsympy(expr, outputs, component):
    inputs = list(expr.free_symbols)
    inpunitsflat = tuple(inpvar.varunit for inpvar in inputs)
    fx = sp.lambdify(inputs, [expr1, expr2], anp_math) #anp_math
    outunitsflat = tuple(outvar.varunit for outvar in outputs)
    fx_scaled = fx_with_units(fx, inpunitsflat, outunitsflat)

In [ ]:
def withunits(fx, inputs, inunitmap, outputs, outputs):
    inpunitsflat = (inunitmap[inpvar] for inpvar in inputs)
    outunitsflat = (outputs[outvar] for outvar in outputs)
    fx_scaled = fx_with_units(fx, inpunitsflat, outunitsflat)